# VCP Pattern Scanner - Stock Market Technical Analysis

This notebook scans for **Vertical Consolidation Pattern (VCP)** opportunities across multiple global markets:
- 🇺🇸 **US Markets**: S&P 500 + Nasdaq (508 stocks)
- 🇭🇰 **Hong Kong**: HSI + HSTECH Combined (103 stocks)
- 🇯🇵 **Japan**: MSCI Japan Index (180 stocks)
- 🔧 **Tech Sector**: Global tech stocks

**VCP Pattern**: Progressive wave contractions forming a tightening range, indicating potential breakout setup.

---

## 1️⃣ Setup: Install Dependencies

In [ ]:
# Install required packages
import subprocess
import sys

packages = ['yfinance', 'pandas', 'numpy', 'requests', 'rich', 'openpyxl']
print("Installing dependencies...")
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + packages)
print("✅ Dependencies installed!")

## 2️⃣ Download: VCP Scanner & Watchlists from GitHub

In [ ]:
import os

# Create directories
os.makedirs('watchlists', exist_ok=True)
os.makedirs('Reports', exist_ok=True)

print("Downloading VCP_scanner.py...")
os.system('wget -q https://raw.githubusercontent.com/clementwai-sketch/Spring/main/VCP_Scanner/VCP_scanner.py')

# Download watchlists
base_url = 'https://raw.githubusercontent.com/clementwai-sketch/Spring/main/VCP_Scanner/watchlists/'
watchlists = ['hk_stocks.json', 'us_stocks.json', 'japan_stocks.json', 'tech_sector.json']

print("Downloading watchlist files...")
for watchlist in watchlists:
    os.system(f'wget -q {base_url}{watchlist} -O watchlists/{watchlist}')
    print(f"  ✅ {watchlist}")

print("\n✅ All files downloaded! Files available:")
os.system('ls -lh VCP_scanner.py watchlists/')

## 3️⃣ Select: Choose Your Watchlist

In [ ]:
# Select which watchlist to scan
print("Choose a watchlist to scan:")
print("  1) Tech Sector (Global Tech Stocks)")
print("  2) US Markets (S&P 500 + Nasdaq)")
print("  3) Japan MSCI (180 Stocks)")
print("  4) Hong Kong (HSI + HSTECH Combined)")
print()

# Store selection for next cell - change this number (1-4) to select different watchlist
WATCHLIST_CHOICE = 1  # Change this to 1, 2, 3, or 4
print(f"Selected: Option {WATCHLIST_CHOICE}")

## 4️⃣ Scan: Run VCP Pattern Detection

In [ ]:
import subprocess
import sys

print(f"Running VCP scanner (Option {WATCHLIST_CHOICE})...")
print("="*60)

# Run scanner with selected watchlist option
result = subprocess.run(
    [sys.executable, 'VCP_scanner.py'],
    input=f"{WATCHLIST_CHOICE}\n",
    text=True,
    capture_output=False
)

print("="*60)
print("✅ Scan complete!")

## 5️⃣ Results: View & Export Findings

In [ ]:
import os
from datetime import datetime

# List generated reports
print("📊 Generated Reports:")
print("="*60)

if os.path.exists('Reports'):
    files = sorted(os.listdir('Reports'), reverse=True)
    if files:
        for file in files[:5]:  # Show last 5 reports
            filepath = os.path.join('Reports', file)
            size = os.path.getsize(filepath) / 1024  # KB
            print(f"  📄 {file} ({size:.1f} KB)")
    else:
        print("  No reports generated yet. Try running the scanner above.")
else:
    print("  Reports directory not found.")

print("\n💡 Tip: Open .csv files with pandas for analysis, .xlsx for spreadsheet view")

In [ ]:
# Load and display latest CSV results
import pandas as pd
import os

csv_files = [f for f in os.listdir('Reports') if f.endswith('.csv')]
if csv_files:
    latest_csv = sorted(csv_files, reverse=True)[0]
    filepath = os.path.join('Reports', latest_csv)
    print(f"📈 Latest Results: {latest_csv}\n")
    
    df = pd.read_csv(filepath)
    print(f"Found {len(df)} VCP patterns")
    print("\n" + "="*60)
    # Display prime VCP patterns (quality > 70) first
    if 'Quality Score' in df.columns:
        prime = df[df['Quality Score'] > 70].sort_values('Quality Score', ascending=False)
        if len(prime) > 0:
            print(f"🎯 PRIME VCP PATTERNS (Quality > 70): {len(prime)}")
            print(prime[[col for col in df.columns if col not in ['Analysis', 'Details']]].head(10).to_string())
    print("\n📊 Full dataset available in df variable")
else:
    print("No CSV files found. Run the scanner cell above first.")

## 6️⃣ Advanced: Configuration Reference

### VCP Detection Parameters

| Parameter | Value | Purpose |
|-----------|-------|----------|
| **MIN_CONTRACTIONS** | 3 | Minimum wave contractions required |
| **MAX_CONTRACTIONS** | 6 | Maximum waves before pattern breaks |
| **CONTRACTION_RATIO** | 0.70 | Each wave must be ≤70% of previous |
| **MAX_LAST_CONTRACTION** | 12.0% | Final squeeze tightness threshold |
| **MIN_PRICE_ABOVE_MA** | 3.0% | Uptrend confirmation (price > 50-day MA) |
| **TREND_MA_PERIOD** | 50 | Moving average for trend filter |
| **VOLUME_MA_PERIOD** | 50 | Moving average for volume analysis |

### Market-Specific Settings

| Market | Zigzag % | Benchmark |
|--------|----------|----------|
| **US** (SPX, NQ) | 5.0% | ^GSPC, ^IXIC |
| **Japan** (.T suffix) | 4.0% | ^N225 |
| **Hong Kong** (.HK suffix) | 6.0% | ^HSI |
| **China** (.SS, .SZ) | 7.0% | FXI |

### Quality Score Breakdown

- **Contractions** (25%): Number and progression of waves
- **Trendiness** (20%): Price positioning vs 50-day MA
- **Volume** (20%): Average volume and recent trends
- **Tightness** (15%): How compressed the final wave is
- **Relative Strength** (20%): Performance vs market benchmark

Score above **70** = Prime Setup | Score 50-70 = Watch List

---

## 📚 Documentation

**GitHub Repository**: [clementwai-sketch/Spring](https://github.com/clementwai-sketch/Spring)

**Files**:
- `VCP_scanner.py` - Main detection engine
- `watchlists/hk_stocks.json` - Hong Kong stocks
- `watchlists/us_stocks.json` - US market (SPX + NQ)
- `watchlists/japan_stocks.json` - Japan MSCI
- `watchlists/tech_sector.json` - Global tech stocks

**Output**:
- `Reports/VCP_Report_*.csv` - Detection results with quality scores
- `Reports/VCP_Report_*.xlsx` - Formatted spreadsheet with charts

**Note**: This is a technical analysis tool for educational purposes. Always perform your own due diligence before trading.

# VCP Pattern Scanner - Stock Market Technical Analysis

This notebook scans for **Vertical Consolidation Pattern (VCP)** opportunities across multiple global markets:
- 🇺🇸 **US Markets**: S&P 500 + Nasdaq (508 stocks)
- 🇭🇰 **Hong Kong**: HSI + HSTECH Combined (103 stocks)
- 🇯🇵 **Japan**: MSCI Japan Index (180 stocks)
- 🔧 **Tech Sector**: Global tech stocks

**VCP Pattern**: Progressive wave contractions forming a tightening range, indicating potential breakout setup.

---

## 1️⃣ Setup: Install Dependencies

In [ ]:
# Install required packages
import subprocess
import sys

packages = ['yfinance', 'pandas', 'numpy', 'requests', 'rich', 'openpyxl']
print("Installing dependencies...")
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + packages)
print("✅ Dependencies installed!")

## 2️⃣ Download: VCP Scanner & Watchlists from GitHub

In [ ]:
import os

# Create directories
os.makedirs('watchlists', exist_ok=True)
os.makedirs('Reports', exist_ok=True)

print("Downloading VCP_scanner.py...")
os.system('wget -q https://raw.githubusercontent.com/clementwai-sketch/Spring/main/VCP_Scanner/VCP_scanner.py')

# Download watchlists
base_url = 'https://raw.githubusercontent.com/clementwai-sketch/Spring/main/VCP_Scanner/watchlists/'
watchlists = ['hk_stocks.json', 'us_stocks.json', 'japan_stocks.json', 'tech_sector.json']

print("Downloading watchlist files...")
for watchlist in watchlists:
    os.system(f'wget -q {base_url}{watchlist} -O watchlists/{watchlist}')
    print(f"  ✅ {watchlist}")

print("\n✅ All files downloaded! Files available:")
os.system('ls -lh VCP_scanner.py watchlists/')

## 3️⃣ Select: Choose Your Watchlist

In [ ]:
# Select which watchlist to scan
print("Choose a watchlist to scan:")
print("  1) Tech Sector (Global Tech Stocks)")
print("  2) US Markets (S&P 500 + Nasdaq)")
print("  3) Japan MSCI (180 Stocks)")
print("  4) Hong Kong (HSI + HSTECH Combined)")
print()

# Store selection for next cell - change this number (1-4) to select different watchlist
WATCHLIST_CHOICE = 1  # Change this to 1, 2, 3, or 4
print(f"Selected: Option {WATCHLIST_CHOICE}")

## 4️⃣ Scan: Run VCP Pattern Detection

In [ ]:
import subprocess
import sys

print(f"Running VCP scanner (Option {WATCHLIST_CHOICE})...")
print("="*60)

# Run scanner with selected watchlist option
result = subprocess.run(
    [sys.executable, 'VCP_scanner.py'],
    input=f"{WATCHLIST_CHOICE}\n",
    text=True,
    capture_output=False
)

print("="*60)
print("✅ Scan complete!")

## 5️⃣ Results: View & Export Findings

In [ ]:
import os
from datetime import datetime

# List generated reports
print("📊 Generated Reports:")
print("="*60)

if os.path.exists('Reports'):
    files = sorted(os.listdir('Reports'), reverse=True)
    if files:
        for file in files[:5]:  # Show last 5 reports
            filepath = os.path.join('Reports', file)
            size = os.path.getsize(filepath) / 1024  # KB
            print(f"  📄 {file} ({size:.1f} KB)")
    else:
        print("  No reports generated yet. Try running the scanner above.")
else:
    print("  Reports directory not found.")

print("\n💡 Tip: Open .csv files with pandas for analysis, .xlsx for spreadsheet view")

In [ ]:
# Load and display latest CSV results
import pandas as pd
import os

csv_files = [f for f in os.listdir('Reports') if f.endswith('.csv')]
if csv_files:
    latest_csv = sorted(csv_files, reverse=True)[0]
    filepath = os.path.join('Reports', latest_csv)
    print(f"📈 Latest Results: {latest_csv}\n")
    
    df = pd.read_csv(filepath)
    print(f"Found {len(df)} VCP patterns")
    print("\n" + "="*60)
    # Display prime VCP patterns (quality > 70) first
    if 'Quality Score' in df.columns:
        prime = df[df['Quality Score'] > 70].sort_values('Quality Score', ascending=False)
        if len(prime) > 0:
            print(f"🎯 PRIME VCP PATTERNS (Quality > 70): {len(prime)}")
            print(prime[[col for col in df.columns if col not in ['Analysis', 'Details']]].head(10).to_string())
    print("\n📊 Full dataset available in df variable")
else:
    print("No CSV files found. Run the scanner cell above first.")

## 6️⃣ Advanced: Configuration Reference

### VCP Detection Parameters

| Parameter | Value | Purpose |
|-----------|-------|----------|
| **MIN_CONTRACTIONS** | 3 | Minimum wave contractions required |
| **MAX_CONTRACTIONS** | 6 | Maximum waves before pattern breaks |
| **CONTRACTION_RATIO** | 0.70 | Each wave must be ≤70% of previous |
| **MAX_LAST_CONTRACTION** | 12.0% | Final squeeze tightness threshold |
| **MIN_PRICE_ABOVE_MA** | 3.0% | Uptrend confirmation (price > 50-day MA) |
| **TREND_MA_PERIOD** | 50 | Moving average for trend filter |
| **VOLUME_MA_PERIOD** | 50 | Moving average for volume analysis |

### Market-Specific Settings

| Market | Zigzag % | Benchmark |
|--------|----------|----------|
| **US** (SPX, NQ) | 5.0% | ^GSPC, ^IXIC |
| **Japan** (.T suffix) | 4.0% | ^N225 |
| **Hong Kong** (.HK suffix) | 6.0% | ^HSI |
| **China** (.SS, .SZ) | 7.0% | FXI |

### Quality Score Breakdown

- **Contractions** (25%): Number and progression of waves
- **Trendiness** (20%): Price positioning vs 50-day MA
- **Volume** (20%): Average volume and recent trends
- **Tightness** (15%): How compressed the final wave is
- **Relative Strength** (20%): Performance vs market benchmark

Score above **70** = Prime Setup | Score 50-70 = Watch List

---

## 📚 Documentation

**GitHub Repository**: [clementwai-sketch/Spring](https://github.com/clementwai-sketch/Spring)

**Files**:
- `VCP_scanner.py` - Main detection engine
- `watchlists/hk_stocks.json` - Hong Kong stocks
- `watchlists/us_stocks.json` - US market (SPX + NQ)
- `watchlists/japan_stocks.json` - Japan MSCI
- `watchlists/tech_sector.json` - Global tech stocks

**Output**:
- `Reports/VCP_Report_*.csv` - Detection results with quality scores
- `Reports/VCP_Report_*.xlsx` - Formatted spreadsheet with charts

**Note**: This is a technical analysis tool for educational purposes. Always perform your own due diligence before trading.

# VCP Pattern Scanner

**Vertical Consolidation Pattern (VCP) Detection System**

Scan stocks for VCP patterns using Mark Minervini's technical analysis methodology.

- 🔍 **Swing Detection**: Zig Zag algorithm (market-specific thresholds)
- 📊 **Pattern Analysis**: Progressive contractions with quality scoring
- 🌍 **Multi-Market**: US, Hong Kong, Japan, China stocks
- 💾 **Smart Cache**: 6-hour expiry for fast re-runs

## Step 1: Install Dependencies

In [ ]:
!pip install yfinance pandas numpy requests rich openpyxl -q
print("✅ Dependencies installed successfully!")

## Step 2: Download VCP_scanner.py from GitHub

In [ ]:
import os

# Get VCP_scanner.py from the repo
!wget -q https://raw.githubusercontent.com/clementwai-sketch/Spring/main/VCP_Scanner/VCP_scanner.py

# Verify
if os.path.exists('VCP_scanner.py'):
    print("✅ VCP_scanner.py downloaded successfully")
    size = os.path.getsize('VCP_scanner.py') / 1024
    print(f"   Size: {size:.1f} KB")
else:
    print("❌ Failed to download VCP_scanner.py")

## Step 3: Download Watchlist Files

In [ ]:
import os
import json

# Create watchlists directory
os.makedirs('watchlists', exist_ok=True)

# List of watchlist files to download
watchlists = [
    'hk_stocks.json',
    'us_stocks.json',
    'japan_stocks.json',
    'tech_sector.json'
]

base_url = 'https://raw.githubusercontent.com/clementwai-sketch/Spring/main/VCP_Scanner/watchlists/'

print("📥 Downloading watchlist files...\n")
for watchlist in watchlists:
    url = base_url + watchlist
    output_path = f'watchlists/{watchlist}'
    
    !wget -q {url} -O {output_path}
    
    if os.path.exists(output_path):
        size = os.path.getsize(output_path) / 1024
        print(f"✅ {watchlist:<20} ({size:.1f} KB)")
    else:
        print(f"❌ {watchlist:<20} (Failed)")

print("\n✓ All watchlist files ready!")

## Step 4: Select Watchlist

Choose which market to scan:
1. **hk_stocks** - Hong Kong (HSI + TECH)
2. **us_stocks** - USA (S&P 500 + Nasdaq-100)
3. **japan_stocks** - Japan (MSCI Japan)
4. **tech_sector** - Multi-market tech stocks

In [ ]:
# Select watchlist to use
selected_watchlist = 'hk_stocks.json'  # Change to: us_stocks.json, japan_stocks.json, or tech_sector.json

watchlist_path = f'watchlists/{selected_watchlist}'

# Load and display watchlist info
import json
with open(watchlist_path, 'r') as f:
    watchlist_data = json.load(f)

# Find the market key
for market_key, config in watchlist_data.items():
    if config.get('enabled', True):
        tickers = config.get('tickers', [])
        print(f"📊 Selected: {market_key}")
        print(f"   Name: {config.get('name', 'N/A')}")
        print(f"   Tickers: {len(tickers)}")
        print(f"   Sample: {', '.join(tickers[:5])}...")

## Step 5: Run VCP Scanner

This will:
1. Fetch historical price data for all tickers
2. Detect VCP patterns
3. Score by quality and relative strength
4. Display results

⏱️ **Runtime**: 2-5 minutes (depends on number of tickers)

In [ ]:
import subprocess
import sys

# Prepare command
cmd = [
    sys.executable,
    'VCP_scanner.py'
]

print("🚀 Starting VCP Pattern Scanner...\n")
print("="*80)

# Run scanner (non-interactive mode - use default watchlist)
result = subprocess.run(cmd, capture_output=False, text=True, input='1\n')

print("="*80)
print("\n✅ Scan complete!")

## Step 6: View Results

Check the generated reports in the `Report/` directory

In [ ]:
import os
from datetime import datetime

# Check for generated reports
report_dir = 'Report'
if os.path.exists(report_dir):
    files = os.listdir(report_dir)
    today = datetime.now().strftime("%Y-%m-%d")
    
    print(f"📁 Report files for {today}:\n")
    for file in files:
        filepath = os.path.join(report_dir, file)
        size = os.path.getsize(filepath) / 1024
        print(f"  📄 {file:<30} ({size:.1f} KB)")
        
        # Show download link for HTML
        if file.endswith('.html'):
            print(f"     → Open in browser for interactive view")
else:
    print("No reports generated yet. Run the scanner first.")

## Advanced: Custom Analysis

Modify the VCP settings before running:

In [ ]:
# View current configuration
print("Current VCP Detection Settings:")
print("="*50)
print(f"MIN_CONTRACTIONS: 3 (minimum waves)")
print(f"CONTRACTION_RATIO: 0.70 (≤70% of previous)")
print(f"MAX_LAST_CONTRACTION: 12.0% (final squeeze)")
print(f"MIN_PRICE_ABOVE_MA: 3.0% (uptrend filter)")
print(f"MIN_AVG_VOLUME: 500,000 (liquidity filter)")
print(f"RS_PERIOD: 63 days (relative strength)")
print("="*50)
print("\nTo adjust: Edit VCP_scanner.py configuration section")

## Documentation

**VCP Pattern Components:**
- **Wave/Contraction**: Peak-to-trough price movement
- **Progressive Narrowing**: Each wave smaller than previous (~70% rule)
- **Volume Drying Up**: Lower volume during contractions
- **Trend Context**: Price above 50-MA (uptrend)
- **Relative Strength**: Stock outperforming benchmark

**Quality Score (0-100):**
- Contractions: More progressive waves = better
- Trend: Higher above MA = better
- Volume: Declining through pattern = better
- Tightness: Tighter final contraction = better
- RS: Outperforming market = better

**Stock Status:**
- 🚀 **BREAKOUT**: Volume spike above pivot
- ⚠️ **VCP FORMING**: Pattern detected, awaiting breakout
- ⭐ **PRIME**: Quality ≥70%, RS63 >5%, RS126 >0%, <10% to pivot

---

**GitHub**: https://github.com/clementwai-sketch/Spring  
**Strategy**: Mark Minervini's Trend Template (VCP variations)